In [2]:
X_train.shape

(34721, 168)

In [19]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from nbeats_keras.model import NBeatsNet as NBeatsKeras
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
#from nbeats_pytorch.model import NBeatsNet as NBeatsPytorch
from keras.optimizers import RMSprop, Adam
import time
from keras.models import load_model
#from target_data_electronic70_7 import target_X, target_y ,test_X, test_y
#from m4databasis21_7 import base_domain,zt_in,zt_out,M4Meta,inputsize,train_12,train_12_y
from sklearn.metrics import mean_squared_error,mean_absolute_error,mean_absolute_percentage_error,r2_score
from tensorflow.keras.losses import Loss
import tensorflow as tf
#from m4databasis35_7_70_7 import train_35,train_35_y,train_70,train_70_y
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, LayerNormalization, MultiHeadAttention, Dropout, Add, Concatenate,Flatten,Reshape
#import pandas as pd
from keras.utils import custom_object_scope

#####################################################################################
X_train = pd.read_csv("../task_transfer_solor_24/data/M4_train.csv").iloc[:,(1):].values
y_train = pd.read_csv("../task_transfer_solor_24/data/M4_test.csv").iloc[:,1:].values
X_train.shape[1], y_train.shape[1]

target_X= pd.read_csv("high_X.csv").iloc[:,(1+24*0):].values
target_y =pd.read_csv("high_y.csv").iloc[:,1:].values
#test_X= pd.read_csv("../data/solor_val_input_7.csv").iloc[:,(1+24*0):].values
#test_y =pd.read_csv("../data/solor_val_output_7.csv").iloc[:,1:].values

from sklearn.model_selection import train_test_split


target_X,test_X,target_y,test_y=train_test_split(target_X,target_y,random_state=1, shuffle=False)





#X_train=target_X
#y_train=target_y

backcast_length = X_train.shape[1]
forecast_length = y_train.shape[1]

X_train.shape,test_X.shape,y_train.shape,test_y.shape

target_X.shape,test_X.shape
#################################################################################
# loss SMAPE
class SMAPE(Loss):
    def call(self, y_true, y_pred):
        y_pred = tf.reshape(y_pred, tf.shape(y_true))  # 예측 값의 차원을 맞춤
       # y_pred=tf.clip_by_value(y_pred, 1e-10, tf.reduce_max(y_pred))
       # y_true = tf.clip_by_value(y_true, 1e-10, tf.reduce_max(y_true))
        
        numerator = 100 * tf.abs(y_true- y_pred )
        denominator =  (tf.abs(y_true ) + tf.abs(y_pred))/2
        smape =  numerator /  denominator #tf.clip_by_value(denominator, 1e-10, tf.reduce_max(denominator))
        return tf.reduce_mean(smape)

#################################################################################
# loss MASE
class MASE(Loss):
    def __init__(self, training_data, period, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.scale = self.calculate_scale(training_data, period)
    def seasonal_diff(data, period):
        return data[period:] - data[:-period]

    def calculate_scale(self, training_data, period):
        # 주기 차분 계산
        diff = seasonal_diff(training_data, period)
        scale = np.mean(np.abs(diff))
        return scale
    
    def call(self, y_true, y_pred):
        y_pred = tf.reshape(y_pred, tf.shape(y_true))  # 차원 맞추기
        error = tf.abs(y_true - y_pred)
        return tf.reduce_mean(error / self.scale)

def seasonal_diff(data, period):
    return data[period:] - data[:-period]

#################################################################################
#################################################################################
# 하이퍼파라미터 인자 설정
def hyperparameter():
    # 1 backcast
    # 2 forecast
    # 3 inputdim
    # 4 outputdim
    # 5 unit
    # 6 bacth size
    return X_train.shape[1], y_train.shape[1],1,1,128

#################################################################################
# nbeats + I모델 생성 함수
def bulid_model(backcast_,forecast_,input_dim,output_dim,unit):
    model= NBeatsKeras(backcast_length=backcast_, 
                       forecast_length=forecast_,
                       input_dim=input_dim,
                       output_dim=output_dim,
                       stack_types=(NBeatsKeras.TREND_BLOCK,
                                    NBeatsKeras.TREND_BLOCK,
                                    NBeatsKeras.TREND_BLOCK,
                                    NBeatsKeras.SEASONALITY_BLOCK,
                                    NBeatsKeras.SEASONALITY_BLOCK,
                                    NBeatsKeras.SEASONALITY_BLOCK)
                   ,nb_blocks_per_stack=1, thetas_dim=(1,2,2,4,4,4),
                   share_weights_in_stack=True, hidden_layer_units=unit)
    return model 
#################################################################################
# nbeats + G모델 생성 함수    
def bulid_model_G(backcast_,forecast_,input_dim,output_dim,unit):
    model= NBeatsKeras(backcast_length=backcast_, 
                       forecast_length=forecast_,
                       input_dim=input_dim,
                       output_dim=output_dim,
                       stack_types=(NBeatsKeras.GENERIC_BLOCK,NBeatsKeras.GENERIC_BLOCK)
                   ,nb_blocks_per_stack=5, thetas_dim=(4,4),
                   share_weights_in_stack=False, hidden_layer_units=unit)
    return model 
#################################################################################
# nbeats + I모델 부트스트랩 샘플링 배깅

def train_bagging_models(num_models, loss_fn , epochs_, patience_,batch_size_,lr):
    models = {}
    backcast,forecast,in_dim,out_dim,unit = hyperparameter()
    historys = []
    for n in range(num_models):
        K.clear_session()
        model = bulid_model(backcast,forecast,in_dim,out_dim,unit)
       # model.set_weights(pretrained_weights)  # 전이 학습 가중치 적용
        optimizer = Adam(learning_rate=lr)
        model.compile(optimizer=optimizer , loss=loss_fn)
        
        # 부트스트랩 샘플링
        select = np.random.choice(len(X_train), size=len(X_train), replace=True)
        X_bootstrap = X_train[select]
        y_bootstrap = y_train[select]
        early_stop = EarlyStopping(monitor='val_loss', patience = patience_, verbose = 0, restore_best_weights=True)
        history = model.fit(X_bootstrap, y_bootstrap, batch_size = batch_size_,
                  epochs=epochs_, verbose=0, 
                  callbacks=[early_stop],
                 validation_split = 0.2)
        models[f'model_{n+1}'] = model
        historys.append(history)
        #models.append(model)
        print(f"'########################################################Model{n}")
    return models,historys
#################################################################################
# nbeats + I모델 부트스트랩 샘플링 배깅

def train_bagging_models_G(num_models, loss_fn , epochs_, patience_,batch_size_,lr):
    models = {}
    backcast,forecast,in_dim,out_dim,unit = hyperparameter()
    historys = []
    for n in range(num_models):
        K.clear_session()
        model = bulid_model_G(backcast,forecast,in_dim,out_dim,unit)
       # model.set_weights(pretrained_weights)  # 전이 학습 가중치 적용
        optimizer = Adam(learning_rate=lr)
        model.compile(optimizer=optimizer , loss=loss_fn)
        
        # 부트스트랩 샘플링
        select = np.random.choice(len(X_train), size=len(X_train), replace=True)
        X_bootstrap = X_train[select]
        y_bootstrap = y_train[select]
        early_stop = EarlyStopping(monitor='val_loss', patience = patience_, restore_best_weights=True)
        history = model.fit(X_bootstrap, y_bootstrap, batch_size = batch_size_,
                  epochs=epochs_, verbose=0, 
                  callbacks=[early_stop],
                 validation_split = 0.2)
        models[f'model_{n+1}'] = model
        historys.append(history)
        #models.append(model)
        print(f"'########################################################Model{n}")
    return models,historys

#################################################################################
##########################################################################################
# 트랜스퍼 레이어
class PositionalEncoding(layers.Layer):
    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = layers.Dropout(rate=dropout)

        position = np.arange(max_len)[:, np.newaxis]
        div_term = np.exp(np.arange(0, d_model, 2) * (-np.log(10000.0) / d_model))
        pe = np.zeros((max_len, d_model))
        pe[:, 0::2] = np.sin(position * div_term)
        pe[:, 1::2] = np.cos(position * div_term)
        pe = pe[np.newaxis, ...]

        self.pe = tf.constant(pe, dtype=tf.float32)

    def call(self, x):
        x = x + self.pe[:, :tf.shape(x)[1], :]
        return self.dropout(x)
##########################################################################################
# 트랜스퍼 레이어
def create_model(fn,d_model, nlayers, nhead, dropout, iw, ow,lr,pretrained_output_reshaped,inputs):
    
    
    x = layers.Dense(d_model // 2, activation='relu')(pretrained_output_reshaped)
    x = layers.Dense(d_model, activation='relu')(x)
    
    pos_encoding = PositionalEncoding(d_model, dropout)
    x = pos_encoding(x)
    
    for _ in range(nlayers):
        attn_output = layers.MultiHeadAttention(num_heads=nhead, key_dim=d_model, dropout=dropout)(x, x)
        x = layers.LayerNormalization(epsilon=1e-6)(x + attn_output)
        ffn_output = layers.Dense(d_model, activation='relu')(x)
        ffn_output = layers.Dense(d_model)(ffn_output)
        x = layers.LayerNormalization(epsilon=1e-6)(x + ffn_output)
    
    x = layers.Dense(d_model // 2, activation='relu')(x)
    x = layers.Dense(1)(x)
    x = tf.squeeze(x, axis=-1)
    
    outputs = layers.Dense((iw + ow) // 2, activation='relu')(x)
    outputs = layers.Dense(ow)(outputs)
    
    optimizer = Adam(learning_rate=lr)
    target_model = Model(inputs=inputs, outputs=outputs)
    target_model.compile(optimizer=optimizer, loss=fn)
    
    return target_model
########################################################################################################################
def transfer_(model_num,models,trainable,
              lossf,epochs_,batch_size_,pt,lr_):
    #model_mapes_G = {}
    #history_mapes_G = []
    model_pred = []
    for i in range(1, model_num+1):
        K.clear_session()
        model_name = f'model_{i}'
        m,_ = models#[model_name]
        model1= m[model_name]
        # 모든 레이어를 학습 불가능하게 설정
        for layer in model1.layers[:-1]:  # 새로운 레이어 추가된 부분은 학습 가능하도록 설정
            layer.trainable = trainable
        pretrained_layers = model1.layers[:-1]
        
        pretrained_model = Model(inputs=model1.input, outputs=pretrained_layers[-1].output)
        
        inputs = Input(shape=(X_train.shape[1], 1))
        pretrained_output = pretrained_model(inputs)
        normalized_output = layers.BatchNormalization()(pretrained_output)

        pretrained_output_reshaped = layers.Reshape((y_train.shape[1], -1))(normalized_output)
            
        model_instance = create_model(lossf ,d_model=64, nlayers=1,nhead=1, dropout=0.1, iw=X_train.shape[1], ow=y_train.shape[1],lr=lr_
                                      ,pretrained_output_reshaped=pretrained_output_reshaped,inputs=inputs)
        early_stop = EarlyStopping(monitor='val_loss', patience=pt, verbose=0, restore_best_weights=True)
    
        history = model_instance.fit(target_X, target_y, batch_size = batch_size_,
                      epochs=epochs_, verbose=0, 
                      callbacks=[early_stop],
                     validation_split = 0.2)
        pred = model_instance.predict(test_X)
        pred = pred.reshape(-1,y_train.shape[1])
        model_pred.append(pred)
        #history_mapes_G.append(history)
        #model_mapes_G[f'model_{i}'] =model_instance
        #del model_instance
        print(f"'########################################################fitted{i}")
    return model_pred

def transfer_save(model_num, model_paths, trainable, lossf, epochs_, batch_size_, pt, lr_):
    history_mapes_G = []
    model_pred = []
    model_path = model_paths
    for i in range(1, model_num + 1):
        K.clear_session()
        
        # 저장된 모델 경로에서 모델 불러오기
        #model_path = model_paths[i - 1]  # 각 모델의 경로를 리스트로 받아 처리
        #model_paths = model_path #+ f'_model_G_{i}.h5'
        model1 = load_model(model_path+ f'_model_G_{i}.h5')  # 저장된 모델 불러오기
        print(f"{model_path}_model_G_{i}.h5")
        
        # 모든 레이어를 학습 불가능하게 설정 (필요한 경우)
        for layer in model1.layers[:-1]:  # 마지막 레이어를 제외하고 학습 가능 여부 설정
            layer.trainable = trainable
        
        pretrained_layers = model1.layers[:-1]
        
        # 전이 학습을 위한 새로운 입력과 모델 구성
        pretrained_model = Model(inputs=model1.input, outputs=pretrained_layers[-1].output)
        inputs = Input(shape=(X_train.shape[1], 1))
        pretrained_output = pretrained_model(inputs)
        pretrained_output_reshaped = layers.Reshape((y_train.shape[1], -1))(pretrained_output)
        
        # 새로운 모델 생성 및 전이 학습
        model_instance = create_model(
            lossf, d_model=64, nlayers=1, nhead=1, dropout=0.1, iw=X_train.shape[1], 
            ow=y_train.shape[1], lr=lr_, pretrained_output_reshaped=pretrained_output_reshaped, inputs=inputs
        )
        
        early_stop = EarlyStopping(monitor='val_loss', patience=pt, verbose=0, restore_best_weights=True)
    
        history = model_instance.fit(
            target_X, target_y, batch_size=batch_size_, epochs=epochs_, verbose=0,
            callbacks=[early_stop], validation_split=0.2
        )
        
        pred = model_instance.predict(test_X)
        pred = pred.reshape(-1, y_train.shape[1])
        model_pred.append(pred)
        history_mapes_G.append(history)
        
        print(f"######################################################## fitted {i}")
    
    return model_pred

def transfer_SMAPE(model_num, model_paths, trainable, lossf, epochs_, batch_size_, pt, lr_):
    history_mapes_G = []
    model_pred = []
    model_path = model_paths
    
    for i in range(1, model_num + 1):
        K.clear_session()
        
        with custom_object_scope({'SMAPE': SMAPE}):  # SMAPE 등록
            model1 = load_model(model_path + f'_model_G_{i}.h5')  # 저장된 모델 불러오기
        
        print(f"{model_path}_model_G_{i}.h5")
        
        # 모든 레이어를 학습 불가능하게 설정 (필요한 경우)
        for layer in model1.layers[:-1]:  # 마지막 레이어를 제외하고 학습 가능 여부 설정
            layer.trainable = trainable
        
        pretrained_layers = model1.layers[:-1]
        
        # 전이 학습을 위한 새로운 입력과 모델 구성
        pretrained_model = Model(inputs=model1.input, outputs=pretrained_layers[-1].output)
        inputs = Input(shape=(X_train.shape[1], 1))
        pretrained_output = pretrained_model(inputs)
        pretrained_output_reshaped = layers.Reshape((y_train.shape[1], -1))(pretrained_output)
        
        # 새로운 모델 생성 및 전이 학습
        model_instance = create_model(
            lossf, d_model=64, nlayers=1, nhead=1, dropout=0.1, iw=X_train.shape[1], 
            ow=y_train.shape[1], lr=lr_, pretrained_output_reshaped=pretrained_output_reshaped, inputs=inputs
        )
        
        early_stop = EarlyStopping(monitor='val_loss', patience=pt, verbose=0, restore_best_weights=True)
    
        history = model_instance.fit(
            target_X, target_y, batch_size=batch_size_, epochs=epochs_, verbose=0,
            callbacks=[early_stop], validation_split=0.2
        )
        
        pred = model_instance.predict(test_X)
        pred = pred.reshape(-1, y_train.shape[1])
        model_pred.append(pred)
        history_mapes_G.append(history)
        
        print(f"######################################################## fitted {i}")
    
    return model_pred
#################################################################################
# 예측

def bagging_predict(models, X):
    predictions = np.array([model.predict(X) for model in models.values()])
    return np.median(predictions, axis=0)

def bagging_predict2(models, X):
    predictions = np.array([model.predict(X) for model in models.values()])
    return predictions

In [9]:
#mape_models = train_bagging_models_G(10,'mape',100,10,1024,0.001)
#smape_models = train_bagging_models_G(10,SMAPE(),100,10,1024,0.001)
mase_models = train_bagging_models_G(10,MASE(y_train,y_train.shape[1]),100,10,1024,0.001)
mase_pred = transfer_(10,mase_models,True, MASE(target_y,y_train.shape[1]),1000,8,30,0.00005)


'########################################################Model0
'########################################################Model1
'########################################################Model2
'########################################################Model3
'########################################################Model4
'########################################################Model5
'########################################################Model6
'########################################################Model7
'########################################################Model8
'########################################################Model9
4/4 [==============================] - 0s 4ms/step
'########################################################fitted1
4/4 [==============================] - 0s 4ms/step
'########################################################fitted2
4/4 [==============================] - 0s 3ms/step
'########################################################fitted3
4/4 [=======

In [14]:
mape_pred = transfer_save(10,'../task_transfer_ele_24/train7/mape_models_G/mape',True, 'mape',1000,8,30,0.00005)
mae_pred = transfer_save(10,'../task_transfer_ele_24/train7/mae_models_G/mae',True, 'mae',1000,8,30,0.00005)
mse_pred = transfer_save(10,'../task_transfer_ele_24/train7/mse_models_G/mse',True, 'mse',1000,8,30,0.00005)
smape_pred = transfer_SMAPE(10,'../task_transfer_ele_24/train7/smape_models_G/smape',True, 'smape',1000,8,30,0.00005)


../task_transfer_ele_24/train7/mape_models_G/mape_model_G_1.h5
4/4 [==============================] - 0s 4ms/step
######################################################## fitted 1
../task_transfer_ele_24/train7/mape_models_G/mape_model_G_2.h5
4/4 [==============================] - 0s 4ms/step
######################################################## fitted 2
../task_transfer_ele_24/train7/mape_models_G/mape_model_G_3.h5
4/4 [==============================] - 0s 4ms/step
######################################################## fitted 3
../task_transfer_ele_24/train7/mape_models_G/mape_model_G_4.h5
4/4 [==============================] - 0s 4ms/step
######################################################## fitted 4
../task_transfer_ele_24/train7/mape_models_G/mape_model_G_5.h5
4/4 [==============================] - 0s 4ms/step
######################################################## fitted 5
../task_transfer_ele_24/train7/mape_models_G/mape_model_G_6.h5
4/4 [==============================] 

ValueError: in user code:

    File "/home/student/anaconda3/envs/py310/lib/python3.10/site-packages/keras/engine/training.py", line 1249, in train_function  *
        return step_function(self, iterator)
    File "/home/student/anaconda3/envs/py310/lib/python3.10/site-packages/keras/engine/training.py", line 1233, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/student/anaconda3/envs/py310/lib/python3.10/site-packages/keras/engine/training.py", line 1222, in run_step  **
        outputs = model.train_step(data)
    File "/home/student/anaconda3/envs/py310/lib/python3.10/site-packages/keras/engine/training.py", line 1024, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "/home/student/anaconda3/envs/py310/lib/python3.10/site-packages/keras/engine/training.py", line 1082, in compute_loss
        return self.compiled_loss(
    File "/home/student/anaconda3/envs/py310/lib/python3.10/site-packa

In [16]:
smape_pred = transfer_SMAPE(10,'../task_transfer_ele_24/train7/smape_models_G/smape',True, SMAPE(),1000,8,30,0.00005)


../task_transfer_ele_24/train7/smape_models_G/smape_model_G_1.h5
4/4 [==============================] - 0s 4ms/step
######################################################## fitted 1
../task_transfer_ele_24/train7/smape_models_G/smape_model_G_2.h5
4/4 [==============================] - 0s 3ms/step
######################################################## fitted 2
../task_transfer_ele_24/train7/smape_models_G/smape_model_G_3.h5
4/4 [==============================] - 0s 3ms/step
######################################################## fitted 3
../task_transfer_ele_24/train7/smape_models_G/smape_model_G_4.h5
4/4 [==============================] - 0s 3ms/step
######################################################## fitted 4
../task_transfer_ele_24/train7/smape_models_G/smape_model_G_5.h5
4/4 [==============================] - 0s 3ms/step
######################################################## fitted 5
../task_transfer_ele_24/train7/smape_models_G/smape_model_G_6.h5
4/4 [====================

In [33]:
concat_G = np.concatenate([np.array(mase_pred),np.array(mape_pred),np.array(mae_pred),np.array(mse_pred),np.array(smape_pred)])

fin_pred_G = np.median(concat_G,axis=0)

np.sqrt(mean_squared_error(test_y.flatten(),fin_pred_G.flatten())),r2_score(test_y.flatten(),fin_pred_G.flatten())

(3.986486223308427, 0.2770277349331334)

In [34]:
concat_G = np.concatenate([np.array(mase_pred),np.array(mae_pred),np.array(mse_pred)])

fin_pred_G = np.median(concat_G,axis=0)

np.sqrt(mean_squared_error(test_y.flatten(),fin_pred_G.flatten())),r2_score(test_y.flatten(),fin_pred_G.flatten())

(3.972275383459936, 0.282172983297178)

In [21]:
concat_G = np.concatenate([np.array(mase_pred)])

fin_pred_G = np.median(concat_G,axis=0)

np.sqrt(mean_squared_error(test_y.flatten(),fin_pred_G.flatten())),r2_score(test_y.flatten(),fin_pred_G.flatten())

(3.978242221739376, 0.2800148375403604)

In [22]:
concat_G = np.concatenate([np.array(mape_pred)])

fin_pred_G = np.median(concat_G,axis=0)

np.sqrt(mean_squared_error(test_y.flatten(),fin_pred_G.flatten())),r2_score(test_y.flatten(),fin_pred_G.flatten())

(4.673584727598899, 0.0063319114343457494)

In [23]:
concat_G = np.concatenate([np.array(smape_pred)])

fin_pred_G = np.median(concat_G,axis=0)

np.sqrt(mean_squared_error(test_y.flatten(),fin_pred_G.flatten())),r2_score(test_y.flatten(),fin_pred_G.flatten())

(4.710211224007162, -0.00930370596154173)

In [24]:
concat_G = np.concatenate([np.array(mae_pred)])

fin_pred_G = np.median(concat_G,axis=0)

np.sqrt(mean_squared_error(test_y.flatten(),fin_pred_G.flatten())),r2_score(test_y.flatten(),fin_pred_G.flatten())

(3.9815612345780544, 0.27881298169924695)

In [25]:
concat_G = np.concatenate([np.array(mse_pred)])

fin_pred_G = np.median(concat_G,axis=0)

np.sqrt(mean_squared_error(test_y.flatten(),fin_pred_G.flatten())),r2_score(test_y.flatten(),fin_pred_G.flatten())

(3.981429781796751, 0.2788606014488112)

In [9]:

pd.DataFrame(fin_pred_G.reshape(-1,24)).to_csv("../result7_new/NBEATs_G_T/pred_mid_G.csv")
for i in range(10):
    pd.DataFrame(concat_G[i].reshape(-1,24)).to_csv(f"../result7_new/NBEATs_G_T/pred_G{i}.csv")